In [2]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r requirements.txt

# Restart the runtime by ending the process.
exit()

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from google.cloud import storage
import google.auth
from serving.constants import  BUCKET, IMG_SOURCE_PREFIX, HIST_DEST_PREFIX, NUM_BANDS, HIST_BINS_LIST, SCALE, CROP
from serving.common import list_blobs_with_prefix
from serving.hist_training import create_hist_dataset, LstmModel, train_and_evaluate, create_data_sample, get_labels
import logging
import io
import itertools
from google.cloud import storage
import google.auth

In [2]:
# SETUP
credentials, _ = google.auth.default()
bucket_name = BUCKET
directory_prefix = IMG_SOURCE_PREFIX
output_prefix = HIST_DEST_PREFIX
labels = "labels_combined.npy"
batch_size = 6

client = storage.Client()
bucket = client.get_bucket(bucket_name)

logging.basicConfig(filename="cerate_dataset.log",level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

counties = ["Lancaster", "Crawford", "Orleans", "Tulare", "Wharton", "Story", "Canyon", "Kit Carson"]
years = range(2017,2019)
months = [5,7,9]

In [6]:
label_data = np.load(labels, allow_pickle=True)
label_header = np.load("labels_header.npy", allow_pickle=True)
label_df = pd.DataFrame(label_data, columns=label_header)
label_df.head(10)

,commodity_desc,reference_period_desc,year,state_name,county_name,target,source_file
0,CORN,YEAR,2016,ALABAMA,BALDWIN,152.9,USDA_Corn_County_2016.csv
1,CORN,YEAR,2016,ALABAMA,BARBOUR,188.9,USDA_Corn_County_2016.csv
2,CORN,YEAR,2016,ALABAMA,CHEROKEE,92.3,USDA_Corn_County_2016.csv
3,CORN,YEAR,2016,ALABAMA,CLEBURNE,125.0,USDA_Corn_County_2016.csv
4,CORN,YEAR,2016,ALABAMA,COFFEE,104.5,USDA_Corn_County_2016.csv
5,CORN,YEAR,2016,ALABAMA,COLBERT,126.2,USDA_Corn_County_2016.csv
6,CORN,YEAR,2016,ALABAMA,CONECUH,116.7,USDA_Corn_County_2016.csv
7,CORN,YEAR,2016,ALABAMA,COVINGTON,108.1,USDA_Corn_County_2016.csv
8,CORN,YEAR,2016,ALABAMA,CULLMAN,117.2,USDA_Corn_County_2016.csv
9,CORN,YEAR,2016,ALABAMA,DALE,145.0,USDA_Corn_County_2016.csv


In [7]:
dataset, input_shape = create_hist_dataset(counties, list(years), months, "labels_combined.npy", "labels_header.npy")
# dataset = dataset.batch(batch_size)

Reshaped histograms shape: (12, 3, 416)
Labels shape: (12, 1)
Number of filtered combinations: 36


In [3]:
data = create_data_sample()

Data point shape (after batching): (2, 3, 416)
Label shape (after batching): (2,)


2024-10-18 01:11:27.380506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [3]
	 [[{{node Placeholder/_1}}]]
2024-10-18 01:11:27.380890: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [3,3,416]
	 [[{{node Placeholder/_0}}]]


In [9]:
# # Assuming you have a dataset named 'dataset'
# dataset_path = f'gs://{bucket_name}/dataset/training_small'

# # Save the dataset
# dataset.save(dataset_path)

In [10]:
# Check the dataset before batching
for data_point, label in dataset.take(1):
    print("Data point shape (before batching):", data_point.shape)
    print("Label shape (before batching):", label.shape)

# # Now, batch the dataset
# dataset = dataset.batch(batch_size)

# # Check the dataset after batching
# for data_point, label in dataset.take(1):
#     print("Data point shape (after batching):", data_point.shape)
#     print("Label shape (after batching):", label.shape)

Data point shape (before batching): (3, 416)
Label shape (before batching): (1,)


2024-10-18 00:50:10.379180: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [12,1]
	 [[{{node Placeholder/_1}}]]


In [11]:
# Initialize and train the model

model = LstmModel(input_shape, lstm_layers=5, no_units=10,)
trained_model, history = train_and_evaluate(model, dataset, epochs=100)

Epoch 1/100


2024-10-18 00:50:12.536477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [12,1]
	 [[{{node Placeholder/_1}}]]


ValueError: Exception encountered when calling layer 'lstm_model' (type LstmModel).

Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (3, 416)

Call arguments received by layer 'lstm_model' (type LstmModel):
  • inputs=tf.Tensor(shape=(3, 416), dtype=float32)
  • training=True